<a href="https://colab.research.google.com/github/yishairasowsky/Sefaria-Project/blob/master/ML/topic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd

In [0]:
prayer_df = pd.read_csv('/content/prayer-sources.csv', 
                        # encoding="latin-1"
                        )
not_prayer_df = pd.read_csv('/content/not-prayer-sources.csv', 
# not_prayer_df = pd.read_csv('/content/not-prayer-sources-more.csv', 
                            # encoding="latin-1"
                            )

In [847]:
prayer_df.sample(3)

,Ref,En,He
372,Bereishit Rabbah 70:16,NaN,"וְעֵינֵי לֵאָה רַכּוֹת (בראשית כט, יז) , אֲמוֹרָאִי דְּרַבִּי יוֹחָנָן תִּרְגֵּם קוֹדְמוֹי וְעֵי..."
175,Yalkut Shimoni on Torah 392:2,NaN,"ויחל משה מלמד שעמד משה בתפלה עד שהוחלה. אביי אמר עד שחלהו להקב""ה בתפלתו. רבא אמר עד שהתיר לו נדר..."
624,"Sefat Emet, Deuteronomy, Vaetchanan 4:5",NaN,"[מ""ש חז""ל המשמיע קולו מקטני אמנה. קשה מה יתרון אמונה שהש""י שומע קול בלחש או בקול שניהם שוים. וי""..."


In [848]:
prayer_df['is_prayer'] = 'yes'
prayer_df = prayer_df[['En','is_prayer']]

prayer_df.tail(3)

,En,is_prayer
1049,"It is true that you are the one who judges, and reproves, who knows all, and bears witness, who ...",yes
1050,"In these Berachot (blessings) we bow down: the first Beracha (blessing) [of Shemoneh Esreh], in ...",yes
1051,On seeds he says [the Beracha (blessing)] Borei Minei Zerayim (Who created different types of se...,yes


In [849]:
not_prayer_df.sample(3)

,Ref,En,He,Topics
259,Menachot 55b:12,"The baraita asks: Do you say that it is singled out for this purpose, to teach that even after t...","אתה אומר לכך יצאת או אינו אלא שזה טעון צפון ואין אחר טעון צפון ת""ל (ויקרא ד, כט) ושחט את החטאת ב...",slaughter-of-sacrifices
853,"Ohr Hashem, Third Treatise, Section One, Fourth Principle 2:1",NaN,הפרק הב' בתכלית המכוון בנס הנפלא הזה ותועלותיו: ולפי שהתכלית המשותף לכל הנסים והנפלאות הוא להקנו...,faith|resurrection
969,Or HaChaim on Exodus 15:26:4,"כי אני ה׳ רפאך , ""For I the Lord am your Healer."" The Torah here refers to diseases which are no...",כי אני ה' רופאך . פירוש לצד חולאים שאינם בידי שמים כאומרם ז''ל (כתובות ל'.) הכל בידי שמים חוץ מצ...,sicknesses


In [850]:
not_prayer_df['is_prayer'] = 'no'
not_prayer_df = not_prayer_df[['En','is_prayer']]
not_prayer_df.sample(3)

,En,is_prayer
516,NaN,no
26,NaN,no
599,This brings us to the commentary on our verse. The Lord G'd was very clever in giving this comma...,no


In [851]:
all_prayer_frames = [prayer_df, not_prayer_df]
all_prayer_df = pd.concat(all_prayer_frames)
all_prayer_df.reset_index(inplace=True)
all_prayer_df=all_prayer_df.rename(columns = {'En':'english_text'})


# all_prayer_df.style.set_properties({'align': 'right'})

# all_prayer_df.tail(3)
all_prayer_df.sample(5)
# all_prayer_df.loc[[345]]

,index,english_text,is_prayer
120,120,The Sages taught: One who goes to measure the grain on his threshing floor recites: May it be Yo...,yes
1687,635,"The verse continues: “Therefore the king was very wrathful, and his anger burned in him” (Esther...",no
436,436,19. The Rabbi: Common prayer has many advantages. In the first instance a community will never p...,yes
1637,585,"The mishna relates: An incident occurred where such a case came before Rabbi Yishmael, and he sa...",no
1404,352,Rabbi Yonatan said to him: We have found everywhere in the Torah that the legal status of a pers...,no


In [852]:
del all_prayer_df['index']

all_prayer_df.sample(3)

,english_text,is_prayer
1895,And ye know that with all my power I have served your father.,no
1185,"And Moses went down from the mount unto the people, and sanctified the people; and they washed t...",no
1418,NaN,no


In [853]:
print("Before dropping duplicates",all_prayer_df.shape)
all_prayer_df.drop_duplicates(inplace=True)
print("After dropping duplicates",all_prayer_df.shape)

print("\nNulls before dropping:",all_prayer_df.isnull().sum()[0])
all_prayer_df.dropna(inplace=True)
print("Nulls after dropping:",all_prayer_df.isnull().sum()[0])

# all_prayer_df.isnull().sum()
# all_prayer_df.sample(5)

Before dropping duplicates (2103, 2)
After dropping duplicates (1223, 2)

Nulls before dropping: 2
Nulls after dropping: 0


In [854]:
shape = all_prayer_df.shape
print(f"{shape[0]} rows and {shape[1]} columns")

1221 rows and 2 columns


In [855]:
all_prayer_df['is_prayer'] = all_prayer_df['is_prayer'].map({'no': 0, 'yes': 1})
all_prayer_df.sample(5)

,english_text,is_prayer
1465,"(1) Sins between man and G-d only. For example, denying the existence of G-d, bad thoughts, bad ...",0
995,Some teach that this statement was made on a baraita referring to another topic. It was taught i...,1
1620,"Rav Huna said to Rav Ḥisda: This case involving Rabban Gamliel is different, as he admitted to h...",0
430,"And since it is difficult for the mind to remember all of this, it was necessary for our sages t...",1
1210,And the LORD prepared a great fish to swallow up Jonah; and Jonah was in the belly of the fish t...,0


In [856]:
print('Prayer:', (all_prayer_df.is_prayer.values == 1).sum())
print('Not prayer:', (all_prayer_df.is_prayer.values == 0).sum())

Prayer: 553
Not prayer: 668


In [885]:
import re
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')

# BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

# BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_ \b\d+\b]')
STOPWORDS = set(stopwords.words('english'))

def process_text(text):
    #  text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

# whitelist = set('abcdefghijklmnopqrstuvwxyz ZXCVBNMASDFGHJKLQWERTYUIOP')
# genres['movie_id'] = genres['movie_id'].astype(str)

# def process_text(text):
#     '''
#     1. Keep only letters and spaces
#     2. Remove stopwords
#     3. Return str of clean text words
#     '''
#     #1
#     chars_str = ''.join(filter(whitelist.__contains__, text))
#     #2
#     clean_words = [word.lower() for word in chars_str.split() if word.lower() not in stopwords.words('english')]
#     #3
#     return ' '.join(clean_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [886]:
sample_text = '''aBC.!@#$%^&    c*(בחתו ‎‎‎ או‎ בקרחתו — une...
233 the gns: ac ...
1041    7. a pien p'''
process_text(sample_text)

'abc # c une233 gns ac 1041 7 pien p'

In [887]:
all_prayer_df.sample(5)

,english_text,is_prayer
1960,gemara respond shmuel could said said even accord opinion rabbi akiva rabbi akiva state opinion ...,0
551,gemara ask one realli say discredit loud voic didnt rabbi yoanan say name rabbi shimon bar yoai ...,1
150,refus said unto master wife gen scriptur state elsewh refer ver bless man trusteth lord jer find...,1
1607,mishna one saw peopl run found ownerless anim eg deer crippl broken leg young pigeon yet learn f...,0
1358,thou shalt follow multitud evil neither shalt thou bear wit caus turn asid multitud pervert justic,0


In [888]:
processed_df = all_prayer_df
processed_df['english_text'] = processed_df['english_text'].apply(process_text)
processed_df.head(5)

,english_text,is_prayer
0,anoth abba binyamin statement regard law prayer taught baraita abba binyamin say two peopl enter...,1
1,taught baraita abba binyamin said one prayer fulli heard synagogu state regard king solomon pray...,1
2,addit ravin bar rav adda said rabbi yitzak said one accustom come synagogu come one day holi one...,1
3,similar taught baraita rabbi yishmael ben elisha high priest said yom kippur enter innermost san...,1
4,mean written let prayer unto lord time favor god abund merci answer truth salvat psalm appear in...,1


In [889]:
# all_prayer_df['En'] = all_prayer_df['En'].apply(stemming)
# all_prayer_df.head()
stemmed_df = processed_df
stemmed_df['english_text'] = stemmed_df['english_text'].apply(stemming)
stemmed_df.head()

,english_text,is_prayer
0,anoth abba binyamin statement regard law prayer taught baraita abba binyamin say two peopl enter...,1
1,taught baraita abba binyamin said one prayer fulli heard synagogu state regard king solomon pray...,1
2,addit ravin bar rav adda said rabbi yitzak said one accustom come synagogu come one day holi one...,1
3,similar taught baraita rabbi yishmael ben elisha high priest said yom kippur enter innermost san...,1
4,mean written let prayer unto lord time favor god abund merci answer truth salvat psalm appear in...,1


In [0]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# prayer_words = ' '.join(list(all_prayer_df[stemmed_df['is_prayer'] == 1]['En']))
# prayer_wc = WordCloud(width = 512,height = 512).generate(prayer_words)
# plt.figure(figsize = (10, 8), facecolor = 'k')
# plt.imshow(prayer_wc)
# plt.axis('off')
# plt.tight_layout(pad = 0)
# plt.show()

In [0]:
# not_prayer_words = ' '.join(list(all_prayer_df[all_prayer_df['is_prayer'] == 0]['En']))
# not_prayer_wc = WordCloud(width = 512,height = 512).generate(not_prayer_words)
# plt.figure(figsize = (10, 8), facecolor = 'k')
# plt.imshow(not_prayer_wc)
# plt.axis('off')
# plt.tight_layout(pad = 0)
# plt.show()

In [892]:
df = stemmed_df
X = df.english_text
y = df.is_prayer
print(f"Totals: {X.shape} / {y.shape}")

from sklearn.model_selection import train_test_split

# train, test = train_test_split(movies, random_state=42, test_size=0.30, shuffle=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=56, test_size=0.20, shuffle=True)

print(f"\nTrain: {X_train.shape} / {y_train.shape}")

print(f"\nTest: {X_test.shape} / {y_test.shape}")

Totals: (1221,) / (1221,)

Train: (976,) / (976,)

Test: (245,) / (245,)


In [0]:
# Convert a collection of text documents to a matrix of token counts

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

# vectorizer = CountVectorizer(analyzer=process_text)
# vectorizer = CountVectorizer(ngram_range=(2, 4))
# vectorizer = HashingVectorizer(ngram_range=(2, 4),alternate_sign=False)
vectorizer = TfidfVectorizer(
    # strip_accents='unicode', 
    # analyzer='word', 
    ngram_range=(1,3), 
    # norm='l2', 
    # max_features = 15000
    # max_features = 12000
    max_features = 10000
    # max_features = 8000
    # max_features = 5000
    )

# vectorizer = TfidfVectorizer(ngram_range=(2, 4))
# vectorizer.fit(X_train)
# vectorizer.get_feature_names()


In [924]:
print("Shape of document term matrix")

# X_train_dtm = vectorizer.transform(X_train)
X_train_dtm = vectorizer.fit_transform(X_train)
print(f"Train: {X_train_dtm.shape}")

# prayer_bag_of_words = vectorizer.fit_transform(all_prayer_df['En'])
X_test_dtm = vectorizer.transform(X_test)
print(f"Test: {X_test_dtm.shape}")


Shape of document term matrix
Train: (976, 10000)
Test: (245, 10000)


In [0]:

# print(vectorizer.get_feature_names())


In [926]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, ComplementNB

classifier = MultinomialNB()
# classifier = ComplementNB()

# for Gaussian need array format
# X_train = X_train.toarray()
# X_test = X_test.toarray()
# classifier = GaussianNB()
# classifier = BernoulliNB()

%time classifier.fit(X_train_dtm, y_train)
y_pred_class = classifier.predict(X_test_dtm)
from sklearn import metrics

print(f"\nBreakdown:\n{y_test.value_counts()[0]} non-prayer (0), and\n{y_test.value_counts()[1]} prayer (1)")

null_accuracy = y_test.value_counts().head(1) / len(y_test)
print(f'\nNull accuracy: {null_accuracy[0]}')

non_prayer_passages = y_test.value_counts()[0]
prayer_passages = y_test.value_counts()[1]
print('Manual null accuracy:',(prayer_passages / (prayer_passages + non_prayer_passages)))

print('\nAccuracy:',round(metrics.accuracy_score(y_test, y_pred_class),2))
print('Precision:',round(metrics.precision_score(y_test, y_pred_class),2))
print('Recall:',round(metrics.recall_score(y_test, y_pred_class),2))
print('F1 score:',round(metrics.f1_score(y_test, y_pred_class),2))

print("\nConfusion Matrix:")
print(metrics.confusion_matrix(y_test, y_pred_class))

CPU times: user 2.38 ms, sys: 18 µs, total: 2.4 ms
Wall time: 2.43 ms

Breakdown:
125 non-prayer (0), and
120 prayer (1)

Null accuracy: 0.5102040816326531
Manual null accuracy: 0.4897959183673469

Accuracy: 0.85
Precision: 0.84
Recall: 0.86
F1 score: 0.85

Confusion Matrix:
[[106  19]
 [ 17 103]]


In [927]:
TP = X_test[y_pred_class + y_test == 2]
TN = X_test[y_pred_class + y_test == 0]
FP = X_test[y_pred_class > y_test]
FN = X_test[y_pred_class < y_test]

pd.options.display.max_colwidth = 100

# pd.style.set_properties(**{'text-align': 'left'})

# print("\n*** TP ***")
# print(TP.sample(5))
# print("\n*** TN ***")
# print(TN.sample(5))
print("*** False Positives: Actually NOT prayer, but misclassified as prayer ***\n")
print(FP.sample(10))
print("\n\n*** False Negatives: Actually prayer, but misclassified as NOT prayer ***\n")
print(FN.sample(10))

*** False Positives: Actually NOT prayer, but misclassified as prayer ***

1644    philosoph ethic instruct passag torah acquaint us marvel characterist gd reluct interf man free ...
1479                                                               make mountain way highway shall rai high
1408    name rav rabbi yehuda said futur holi one bless rejoic righteous let shechinah dwell among every...
1120                                             rabbi iyya bar abba said rabbi yoanan said one brother die
1210                            lord prepar great fish swallow jonah jonah belli fish three day three night
1256                                                          fal balanc abomin lord perfect weight delight
1144                                                                        bless god eaten whose good live
1071                                  shall know lord god brought forth land egypt may dwell among lord god
1571    gone citi yet far joseph said unto steward follow men

In [0]:
# FP.to_csv('FP', sep='\t', encoding='utf-8')
# FN.to_csv('FN', sep='\t', encoding='utf-8')
